In [14]:
from tkinter import *
root = Tk();

root.mainloop()

In [145]:
from tkinter import *
import time
import numpy as np
from PIL import Image
from PIL import ImageTk

class App:

    def __init__(self, master):

        self.master = master
        self.defaults = {'step':'20',
                        'overlay':'40',
                        'outletVolume':'30',
                        'outletSize':'50',
                        'outletSpeed':'5',
                        'simulationStep':'0.5',
                        'dropsPerVolumeUnit':'300'}
        master.title('Painter')
        master.geometry('800x500')
        self.create_widgets(master)
        master.mainloop()
        
    def create_widgets(self,master):
        
        self.canv = Canvas(master,width = 500, height = 500, bg = '#fff', cursor='cross')
        self.canv.pack(side = 'right')
        y=10
        lbl1 = Label(master, text = 'Step:', bg='#fff') 
        lbl1.place(x=130,y=y,width = 40)
        stepTextField = Entry(master)
        stepTextField.place(x=180,y=y,width = 100)
        y+=20
        lbl2 = Label(master, text = 'Overlay:', bg='#fff') 
        lbl2.place(x=115,y=y,width = 55)
        overlayTextField = Entry(master)
        overlayTextField.place(x=180,y=y,width = 100)
        y+=20
        lbl3 = Label(master, text = 'Paint outlet volume\n           (particles/s):', bg='#fff') 
        lbl3.place(x=50,y=y,width = 120)
        y+=13
        outletIntensityTextField = Entry(master)
        outletIntensityTextField.place(x=180,y=y,width = 100)
        y+=20
        lbl4 = Label(master, text = 'Drops per volume:', bg='#fff') 
        lbl4.place(x=55,y=y,width = 115)
        dropsPerVolumeUnitField = Entry(master)
        dropsPerVolumeUnitField.place(x=180,y=y,width = 100)
        y+=20
        lbl5 = Label(master, text = 'Paint outlet size:', bg='#fff') 
        lbl5.place(x=70,y=y,width = 100)
        outletSizeTextField = Entry(master)
        outletSizeTextField.place(x=180,y=y,width = 100)
        y+=20
        lbl6 = Label(master, text = 'Paint outlet speed:', bg='#fff') 
        lbl6.place(x=55,y=y,width = 115)
        outletSpeedTextField = Entry(master)
        outletSpeedTextField.place(x=180,y=y,width = 100)
        y+=20
        lbl7 = Label(master, text = 'Simulation step:', bg='#fff') 
        lbl7.place(x=70,y=y,width = 100)
        simulationStepField = Entry(master)
        simulationStepField.place(x=180,y=y,width = 100)
        y+=20
        self.lblFrameCounter = Label(master,text="Frame No:")
        self.lblFrameCounter.place(x=70,y=y,width = 100)
        
        self.envies ={'step':stepTextField,
                      'overlay':overlayTextField,
                      'outletVolume':outletIntensityTextField,
                      'outletSpeed':outletSpeedTextField,
                      'outletSize':outletSizeTextField,
                      'simulationStep':simulationStepField,
                      'dropsPerVolumeUnit':dropsPerVolumeUnitField}
        self.btn = Button(master,text='Try configuration!', command=self.process)
        self.btn.place(x=80,y=240)
        self.btn1 = Button(master,text='Show image', command=self.showImage)
        self.btn1.place(x=80,y=270)
        
        
    def showImage(self):
        image = Image.open("painter.png").rotate(90)
        photo = ImageTk.PhotoImage(image)
        self.image_lbl = Label(image=photo)
        self.image_lbl.image = photo # keep a reference!
        self.image_lbl.place(x=300,y=0)
        
        
    def process(self):
        self.params={}
        for key in self.envies:
            try:
                self.params[key] = float(self.envies[key].get())
            except:
                self.envies[key].delete(0,1000)
                self.envies[key].insert(0,self.defaults[key])
                self.params[key] = float(self.envies[key].get())
        
        self.compute_outlet_line(step=self.params['step'], 
                              overlay=self.params['overlay'])
        self.image_lbl.destroy()
        self.process_outlet(self.outletxy, self.params)
        
        
    def process_outlet(self, xy, params):
        self.bitmap = np.array([[0 for i in range(int(self.canv['width']))]\
                                for i in range(int(self.canv['height']))], dtype = 'float')
        self.canv.create_line(xy, fill='#dddddd') #outlet trace
        x,y = xy[0]
        half_size = int(params['outletSize'])/2 #params of outlet cicle
        speed = params['outletSpeed']*params['simulationStep']
        xy_node_no = 0
        frame=0
        circle = self.canv.create_oval(-half_size,-half_size,+half_size,+half_size)
        
        while(xy_node_no < len(xy)-1):
            speed_vec = self.normalise_vector([xy[xy_node_no+1][0]-
                xy[xy_node_no][0], xy[xy_node_no+1][1]-xy[xy_node_no][1]])
            speed_vec = [speed_vec[0]*speed, speed_vec[1]*speed]
            presicion = 1.2*max(abs(speed_vec[0]),abs(speed_vec[1]))
            while(max(abs(x-xy[xy_node_no+1][0]),abs(y-xy[xy_node_no+1][1])) > presicion):
                cnt_volume=0
                while(int(self.params['dropsPerVolumeUnit'])*self.params['outletVolume']/speed>cnt_volume): #сделать в цикле
                    cnt_volume+=1
                    #begin paint
                    blobpx = int(x+np.random.normal()*half_size/3)#!!!!!!
                    blobpy = int(y+np.random.normal()*half_size/3)
                    if(0<=blobpx<int(self.canv['width'])-2 and 0 <= blobpy < int(self.canv['height'])-2):
                        blob = abs(np.random.normal())
                        self.bitmap[blobpx-1:blobpx+2,blobpy-1:blobpy+2]+=blob
                        self.bitmap[blobpx,blobpy]+=blob/2
                    #end paint
                self.canv.coords(circle,x-half_size,y-half_size,x+half_size,y+half_size)
                x+=speed_vec[0]
                y+=speed_vec[1]
                self.lblFrameCounter['text'] = str(x)+','+str(y)
                self.master.update_idletasks()
                self.master.update()
            xy_node_no+=1
        self.bitmap=self.bitmap/np.max(self.bitmap)
        self.bitmap = 255-255*self.bitmap
        g = np.asarray(dtype=np.dtype('uint8'), a=self.bitmap)
        Image.fromarray(g, mode='L').convert('1').save('painter.png')
        #сделать анализ
        
                
    def normalise_vector(self, v):
        mag = (v[0]**2+v[1]**2)**0.5
        return [v[0]/mag, v[1]/mag]
            
        
    def compute_outlet_line(self, variant = 0, step = 20, overlay = 20):
        if (variant == 0):
            xy=[]
            x = -2*step
            while(x<500+overlay):
                xy.append([x,-overlay])
                x+=step
                xy.append([x,-overlay])
                xy.append([x,500+overlay])
                x+=step
                xy.append([x,500+overlay])
        self.outletxy = xy
        

root = Tk()
app = App(root)
root.destroy() # only for debug

Exception in Tkinter callback
Traceback (most recent call last):
  File "/usr/lib/python3.5/tkinter/__init__.py", line 1553, in __call__
    return self.func(*args)
  File "<ipython-input-145-e24630cfa42c>", line 102, in process
    self.process_outlet(self.outletxy, self.params)
  File "<ipython-input-145-e24630cfa42c>", line 127, in process_outlet
    if(0<=blobpx<int(self.canv['width'])-2 and 0 <= blobpy < int(self.canv['height'])-2):
  File "/usr/lib/python3.5/tkinter/__init__.py", line 1337, in cget
    return self.tk.call(self._w, 'cget', '-' + key)
_tkinter.TclError: invalid command name ".140253240975032"
Exception in Tkinter callback
Traceback (most recent call last):
  File "/usr/lib/python3.5/tkinter/__init__.py", line 1553, in __call__
    return self.func(*args)
  File "<ipython-input-145-e24630cfa42c>", line 102, in process
    self.process_outlet(self.outletxy, self.params)
  File "<ipython-input-145-e24630cfa42c>", line 127, in process_outlet
    if(0<=blobpx<int(self.c

TclError: can't invoke "destroy" command: application has been destroyed

In [63]:
import numpy as np
a = np.array([[0 for i in range(int(500))] for i in range(int(50))], dtype = 'float')
x = np.random.randint(20,40)
y = np.random.randint(30,50)
blob = abs(np.random.normal())
a[x-1:x+2,y-1:y+2]+=blob
a[x,y]+=blob/2
b = np.array((255-a/np.max(a)*255),dtype='int')

255

In [49]:
from PIL import Image
import numpy as np

if __name__ == '__main__':
    g = np.asarray(dtype=np.dtype('uint8'), a=(255-a/np.max(a)*255))
    i = Image.fromarray(g, mode='L').convert('1')
    i.save('painter.png')

[[255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 ...
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]]
